# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download
To: /content/songs_dataset.zip
100%|██████████| 2.09M/2.09M [00:00<00:00, 146MB/s]


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['dolly-parton.txt',
 'cake.txt',
 'britney-spears.txt',
 'paul-simon.txt',
 'beatles.txt',
 'nickelback.txt',
 'rihanna.txt',
 'disney.txt',
 'kanye.txt',
 'drake.txt',
 'kanye-west.txt',
 'jimi-hendrix.txt',
 'nicki-minaj.txt',
 'janisjoplin.txt',
 'alicia-keys.txt',
 'bieber.txt',
 'bob-dylan.txt',
 'amy-winehouse.txt',
 'ludacris.txt',
 'bruno-mars.txt',
 'lin-manuel-miranda.txt',
 'r-kelly.txt',
 'eminem.txt',
 'bob-marley.txt',
 'notorious_big.txt',
 'dr-seuss.txt',
 'dj-khaled.txt',
 'missy-elliott.txt',
 'bruce-springsteen.txt',
 'adele.txt',
 'lady-gaga.txt',
 'patti-smith.txt',
 'joni-mitchell.txt',
 'lil-wayne.txt',
 'lorde.txt',
 'nursery_rhymes.txt',
 'prince.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'bjork.txt',
 'leonard-cohen.txt',
 'notorious-big.txt',
 'Lil_Wayne.txt',
 'nirvana.txt',
 'radiohead.txt',
 'johnny-cash.txt',
 'blink-182.txt',
 'dickinson.txt',
 'Kanye_West.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bruno-mars.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,Now greetings to the world! Standing at this l...
1,"Whiskey coming through my pores,"
2,Feeling like I run this whole block.
3,Lotto tickets cheap beer
4,"That's why you can catch me here,"


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3270


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[427:435]

[['no', 'no', 'no', 'no', 'this', 'hit', 'that', 'ice', 'cold'],
 ['michelle', 'pfeiffer', 'that', 'white', 'gold'],
 ['this', 'one', 'for', 'them', 'hood', 'girls'],
 ['them', 'good', 'girls', 'straight', 'masterpieces'],
 ["stylin'", 'whilen', "livin'", 'it', 'up', 'in', 'the', 'city'],
 ['got', 'chucks', 'on', 'with', 'saint', 'laurent'],
 ['got',
  'kiss',
  'myself',
  "i'm",
  'so',
  'pretty',
  "i'm",
  'too',
  'hot',
  'hot',
  'damn'],
 ['called', 'a', 'police', 'and', 'a', 'fireman']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 3270


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 678


In [13]:
w2v_model.wv.vocab

{"'cause": <gensim.models.keyedvectors.Vocab at 0x7fb75f1d4410>,
 "'til": <gensim.models.keyedvectors.Vocab at 0x7fb75f1d4f10>,
 '3x': <gensim.models.keyedvectors.Vocab at 0x7fb75f1f38d0>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7fb75f1d45d0>,
 'about': <gensim.models.keyedvectors.Vocab at 0x7fb7a5a14f10>,
 'afraid': <gensim.models.keyedvectors.Vocab at 0x7fb75f1e9090>,
 'again': <gensim.models.keyedvectors.Vocab at 0x7fb75f1d88d0>,
 'agree': <gensim.models.keyedvectors.Vocab at 0x7fb75f1e10d0>,
 'ah': <gensim.models.keyedvectors.Vocab at 0x7fb75f1ed890>,
 'ahh': <gensim.models.keyedvectors.Vocab at 0x7fb75f1f34d0>,
 "ain't": <gensim.models.keyedvectors.Vocab at 0x7fb75f1d4590>,
 'aint': <gensim.models.keyedvectors.Vocab at 0x7fb75f1f3350>,
 'air': <gensim.models.keyedvectors.Vocab at 0x7fb7a5a145d0>,
 'all': <gensim.models.keyedvectors.Vocab at 0x7fb75f1de8d0>,
 'alone': <gensim.models.keyedvectors.Vocab at 0x7fb75f1e5d10>,
 'along': <gensim.models.keyedvectors.Vocab at 0x7fb75f1

### 3 - Entrenar el modelo generador

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 127963.234375
Loss after epoch 1: 70639.421875
Loss after epoch 2: 68371.34375
Loss after epoch 3: 68440.0
Loss after epoch 4: 67895.96875
Loss after epoch 5: 67014.09375
Loss after epoch 6: 66072.25
Loss after epoch 7: 65090.4375
Loss after epoch 8: 62438.875
Loss after epoch 9: 60414.4375
Loss after epoch 10: 57853.8125
Loss after epoch 11: 55148.5
Loss after epoch 12: 53142.875
Loss after epoch 13: 50789.9375
Loss after epoch 14: 49241.6875
Loss after epoch 15: 47743.5625
Loss after epoch 16: 43850.6875
Loss after epoch 17: 42326.0
Loss after epoch 18: 41557.875
Loss after epoch 19: 40269.875
Loss after epoch 20: 39518.0
Loss after epoch 21: 38581.5
Loss after epoch 22: 37890.25
Loss after epoch 23: 37237.25
Loss after epoch 24: 36222.0
Loss after epoch 25: 36143.25
Loss after epoch 26: 35741.75
Loss after epoch 27: 35369.125
Loss after epoch 28: 34668.25
Loss after epoch 29: 34222.25
Loss after epoch 30: 34062.75
Loss after epoch 31: 33843.75
Loss after epoch 32

(820625, 1345000)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["liquor"], topn=10)

[('store', 0.8693147301673889),
 ('blues', 0.6674636602401733),
 ('favourite', 0.6395317316055298),
 ('higher', 0.6279494762420654),
 ('pain', 0.6052544116973877),
 ('part', 0.5811115503311157),
 ('shot', 0.5584149360656738),
 ('lovelies', 0.5511866211891174),
 ('voices', 0.5399754047393799),
 ('sorrow', 0.5314211845397949)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["heaven"], topn=10)

[('goodbye', 0.2077248990535736),
 ('say', 0.19475625455379486),
 ('want', 0.18129926919937134),
 ('else', 0.13903313875198364),
 ('ask', 0.10727423429489136),
 ('yeah', 0.10708113014698029),
 ('girl', 0.10660406202077866),
 ('exactly', 0.1060822457075119),
 ('when', 0.10411880165338516),
 ('wait', 0.09912270307540894)]

In [17]:
# Palabras que MAS se relacionan con...:
w2v_model.wv.most_similar(positive=["mom"], topn=10)

[('favorite', 0.7859001755714417),
 ('woman', 0.4942712187767029),
 ('treasure', 0.4903169870376587),
 ('way', 0.4823775887489319),
 ('higher', 0.47283807396888733),
 ('perfect', 0.46092748641967773),
 ('who', 0.4591675400733948),
 ('wanted', 0.45308881998062134),
 ('wish', 0.4426000714302063),
 ('always', 0.43529701232910156)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["diamonds"], topn=10)

[('sheets', 0.9572197198867798),
 ('white', 0.8807447552680969),
 ('silk', 0.8760440945625305),
 ('dim', 0.6828609704971313),
 ('lovelies', 0.6582652926445007),
 ('sweet', 0.6364806890487671),
 ('roll', 0.6294019222259521),
 ('gave', 0.6117922067642212),
 ('lights', 0.5893544554710388),
 ('through', 0.5875629782676697)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["strawberry"], topn=5)

[('champagne', 0.98259437084198),
 ('ice', 0.9004901647567749),
 ('ohhh', 0.8350951075553894),
 ('spell', 0.7247098684310913),
 ('whoo', 0.7098385095596313)]

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["veins"])

[('shoot', 0.9267868399620056),
 ('bed', 0.751532256603241),
 ('hair', 0.7302831411361694),
 ('head', 0.6936258673667908),
 ('brain', 0.691990852355957),
 ('voices', 0.6913931369781494),
 ('eyes', 0.6685410737991333),
 ('favourite', 0.6668327450752258),
 ('tears', 0.6620155572891235),
 ('finesse', 0.6417572498321533)]

### 5 - Visualizar agrupación de vectores

In [21]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab
